<img src='https://upload.wikimedia.org/wikipedia/fr/thumb/e/ed/Logo_Universit%C3%A9_du_Maine.svg/1280px-Logo_Universit%C3%A9_du_Maine.svg.png' width="300" height="500">
<br>
<div style="border: solid 3px #000;">
    <h1 style="text-align: center; color:#000; font-family:Georgia; font-size:26px;">Infrastructures pour l'IA</h1>
    <p style='text-align: center;'>Master Informatique 1</p>
    <p style='text-align: center;'>Anhony Larcher</p>
</div>

Cette session est organisée comme un challenge:
* Vous optimiserez un code afin d'effectuer un calcul le plus rapidement possible
* le résultat sera utilisée dans le TP numéro 6 pour faire du calcul parallélisé.


# Implémenter le calcul suivant et optimisez le au mieux

Il s'agit d'accumuler les statistiques d'ordre 0 et d'ordre 1 sur un mélange de Gaussiennes. Vous trouverez sur Umtice une archive zip contenant un objet Mixture et des paramètres acoustiques de type MFCC.

Chaque distribution de la mixture a pour densité de probabilité:

Et la densité de probabilité du mélange de Gaussienne est 
$$p(x|\lambda) = \Sigma_{i=1}^{M}w_ip_i(x)$$

où $$\Sigma_{i=1}^{M}w_i = 1$$

et 

$$p_i(x) = \frac{1}{(2\pi)^{\frac{D}{2}}|\Sigma_i|^\frac{1}{2}} e^{-(\frac{1}{2}(x - \mu)^T\Sigma_i^{-1}(x-\mu))}$$

Nous souhaiton ici calculer pour chaque trame $x_t$

$$P(i|x_t) = \frac{w_ip_i(x_t)}{\Sigma_{j=1}^M w_j p_j(x_t)}$$

Pour ensuite calculer les statistiques d'ordre 0 pour chaque gaussienne $i$:

$$n_i = \Sigma_{t=1}^{T}P(i|x_t)$$

Les statistiques d'ordre 1:

$$E_i(x) = \frac{1}{n_i} \Sigma_{t=1}^{T}P(i|x_t)x_t$$

Et les statistiques d'ordre 2:

$$E_i(x^2) = \frac{1}{n_i} \Sigma_{t=1}^{T}P(i|x_t)x_t^2$$

Les données sont disponibles ici:

    https://umbox.univ-lemans.fr/index.php/s/CiP47cfw8NBMM5J

## 1.1) Initialisez l'accumulateur de statistiques

Le fichier ``mixture.py``contient le code d'une classe Mixtuyre qui sera utilisé dans ce TP. Ce code se trouve également dans la cellule ci-dessous pour travailler dans le notebook. Une fois la cellule ci-dessous complétée vous pourrez recopier le code dans le fichier ``mixture.py``afin d'avoir une version propre du code.

In [1]:
import numpy
import pickle
import math
import time

class Mixture(object):

    def __init__(self):
        """
        Initialize an empty Mixture object
        """
        self.w = numpy.array([])
        self.mu = numpy.array([])
        self.invcov = numpy.array([])
        self.cst = numpy.array([])
        self.det = numpy.array([])
        self.D = 0 

    def __repr__(self):
        """
        Serialize a Mixture object to text
        """
        return f'w = {self.w.shape}{self.w}\nmu = {self.mu.shape}{self.mu}\ninvcov = {self.invcov.shape}{self.invcov}\ncst = {self.cst.shape}{self.cst}\ndet = {self.det.shape}{self.det}'

    @classmethod
    def read(cls, filename):
        """
        Read a Mixture object stored in Pickle format on disk
        :param filename: the name of the file to read from
        :return: a Mixture object
        """
        with open(filename, 'rb') as fh:
            mixture = pickle.load(fh)
            return mixture

    def save(self, filename):
        """
        Save a Mixture object to disk in Pickle format
        :param filename: the name of the file to write to
        """
        with open(filename, 'wb') as fh:
            pickle.dump(self, fh)

    def loadPresets(self):
        self.mu = numpy.load("mu.npy")
        self.w  = numpy.load("w.npy")
        self.invcov = numpy.load("invcov.npy")
        self.D = self.mu.shape[1]


## Analyse des formules

On remarque d'abord que certains éléments dépendent des données, et plus particulièrement de chaque vecteur de donnée, mais également de chaque distribution de la mixture.

Nous verrons donc apparaitre 2 boucles principales:
* une sur les données
* une sur les distributions

Pour simplifier le calcul nous devons donc séparer ce qui est indépendant de ces boucles afin de ne pas le recalculer plusieurs fois.

Ré-écrivez $$p_i(x)$$ en séparant ces termes.

$$p_i(x) = \frac{1}{(2\pi)^{\frac{D}{2}}|\Sigma_i|^\frac{1}{2}} e^{-(\frac{1}{2}(x - \mu_i)^T\Sigma_i^{-1}(x-\mu_i))}$$

On trouve d'abord un terme qui dépend de chaque mixture: leur déterminant

In [44]:
# Il faut donc calculer ce déterminant une seule fois:
# Les covariances inverses étant stockées par ligne dans self.invcov nous calculons simplement le produit de chaque ligne.
# Le grand Sigma dans notre équation, c'est la matrice de covariance.
# Comme on l'a expliqué en TD, pour simplifier les calculs on ne prend qu'une matrice diagonale, car ça va plus vite à calculer
# Ce qui est chouette c'est que le determinant d'une matrice diagonale est juste le produit de ses termes diagonaux.
# Seul petit détail, on nous a donné l'inverse de la covariance. Il faudra donc la recalculer.
# C'est pas bien gênant, on fait juste 1/v pour chaque valeur, car dans le cas d'une matrice diagonale on peut calculer son inverse comme ça.
#
# Just a little note, in the following code cells of this notebook, I am not going to use the mixture object above, but rather do the code inline and adapt it in the Mixture class.

from time import perf_counter_ns as perf

invcov = numpy.load("invcov.npy")

# With loops and no numpy
start = perf()
det2 = []
for line in invcov:
    det2.append(1)
    for val in line:
        det2[-1] *= val
    det2[-1] = 1 / det2[-1]
end = perf()
naive_imple = end-start
       
# With numpy and no loops 
start = perf()
det = 1.0 / numpy.prod(invcov, axis=1)
end = perf()
numpy_imple = end-start

print(f"Naive implementation took {naive_imple/1000} microseconds\n"
      f"Numpy implementation took {numpy_imple/1000} microseconds\n"
      f"So, numpy here is {naive_imple/numpy_imple:.2f} times faster")

Naive implementation took 2056.4 microseconds
Numpy implementation took 265.3 microseconds
So, numpy here is 7.75 times faster
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


Nous pouvons maintenant calculer le premier terme pour chaque distribution:
    
$$ \frac{1}{(2\pi)^{\frac{D}{2}}|\Sigma_i|^\frac{1}{2}} $$ 

In [51]:
# D étant constant, on voit d'ores et déjà que (2Pi)^(D/2) est une constante également. Le déterminant de la matrice de covariance ne dépend pas des donnés, on peut donc calculer cette fraction en amont, une seule fois pour toutes les données puisqu'elles n'influent pas dans le calcul ici.
# Enfin, n'oublions la racine carré sur le determinant de la covariance.
# Manque plus qu'inverser tout ça et ce sera bon.

import math

D = invcov.shape[1]

### Pure python implementation ###
start = perf()
cst2 = []
for line in det:
    cst2.append( 1.0 / ((2.0 * math.pi)**(D/2.0) * (math.sqrt(line))) )
end = perf()
naive_imple = end-start

### Numpy implementation ###
start = perf()
cst = 1.0 / ( numpy.power(2.0 * numpy.pi, D/2.0) * numpy.sqrt( det ))
end = perf()
numpy_imple = end-start

print(f"Naive implementation took {naive_imple/1000} microseconds\n"
      f"Numpy implementation took {numpy_imple/1000} microseconds\n"
      f"So, numpy here is {naive_imple/numpy_imple:.2f} times faster")


Naive implementation took 352.4 microseconds
Numpy implementation took 194.2 microseconds
So, numpy here is 1.81 times faster


Regardons maintenant le terme contenu dans l'exponentielle:

$$ -(\frac{1}{2}(x - \mu_i)^T\Sigma_i^{-1}(x-\mu_i)) $$ 

Tout ce terme dépend des distributions, une partie seulement dépend des données.

$$ -(\frac{1}{2}(x - \mu_i)^T\Sigma_i^{-1}(x-\mu_i)) = -\frac{1}{2} ( x^T\Sigma_i^{-1}x + \mu_i^T\Sigma_i^{-1}\mu_i - x^T\Sigma_i^{-1}\mu_i - \mu_i^T\Sigma_i^{-1}x ) $$ 

Calculez chacun de ces termes:

Analyse du terme independant des données:

$$ \mu_i^T\Sigma_i^{-1}\mu_i $$

$$ \mu_i^T$$ est de dimension (1, 39) $$\Sigma_i$$ est de dimension (39, 39)

Donc $$ \mu_i^T\Sigma_i^{-1}\mu_i $$ est de dimension (1,)

In [52]:
# En pratique, comme la matrice de covariance est diagonale, nous n'avons pas besoin de calculer un vrai produit matriciel
# Écrivez la valeur du premier terme de \mu_i \Sigma^{-1}_i et tirez partie de cette expression pour simplifier
# le calcul de ce terme

# self.mu est de dimension 64x39
# self.invcov est de dimension 64x39
# numpy.square(self.mu) * self.invcov) est de dimension 64x39
# numpy.square(self.mu) * self.invcov).sum(1) est de dimension 64 
# A est de dimension 64

mu = numpy.load("mu.npy")

### Pure python implementation ###
start = perf()
a_idpt2 = []
for row_invcov, row_mu in zip(invcov, mu):
    a_idpt2.append(0)
    for el_invcov, el_mu in zip(row_invcov, row_mu):
        a_idpt2[-1] += el_mu * el_invcov * el_mu
end = perf()
naive_imple = end-start

### Numpy implementation ###
start = perf()
a_idpt = numpy.sum( numpy.square(mu) * invcov, axis=1 )
end = perf()
numpy_imple = end-start

print(f"Naive implementation took {naive_imple/1000} microseconds\n"
      f"Numpy implementation took {numpy_imple/1000} microseconds\n"
      f"So, numpy here is {naive_imple/numpy_imple:.2f} times faster")

print(a_idpt - numpy.array(a_idpt2))

Naive implementation took 3950.0 microseconds
Numpy implementation took 169.3 microseconds
So, numpy here is 23.33 times faster
[ 0.00000000e+00  7.10542736e-15  0.00000000e+00 -7.10542736e-15
  0.00000000e+00  0.00000000e+00 -3.55271368e-15 -7.10542736e-15
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  1.06581410e-14  7.10542736e-15  0.00000000e+00 -7.10542736e-15
  0.00000000e+00  0.00000000e+00 -7.10542736e-15 -3.55271368e-15
 -3.55271368e-15  1.06581410e-14 -7.10542736e-15 -7.10542736e-15
  1.77635684e-15 -7.10542736e-15 -1.42108547e-14  1.42108547e-14
  0.00000000e+00 -7.10542736e-15  0.00000000e+00  3.55271368e-15
  0.00000000e+00  0.00000000e+00  1.06581410e-14  3.55271368e-15
  0.00000000e+00  7.10542736e-15 -7.10542736e-15  0.00000000e+00
 -3.55271368e-15 -1.06581410e-14 -3.55271368e-15 -3.55271368e-15
  0.00000000e+00  0.00000000e+00  0.00000000e+00  7.10542736e-15
 -3.55271368e-15  0.00000000e+00  3.55271368e-15 -7.10542736e-15
 -3.55271368e-15  0.0000000

Pareil pour le terme $$ x^T\Sigma_i^{-1}x = \sum_{n=1}^{39}{x^2 \sigma_n} $$

In [53]:
X = numpy.load("features/features_1.npy")

# Voici ce que j'avais initialement fait comme implementation.
# Je la garde ici juste pour référence.
# Comme on peut le voir, j'utilisais à la fois des boucles et du numpy. Donc il y a un peu d'optimisation, mais on voit que ça manque un peu quand même.

### My first implementation ###
def f(x):
    return numpy.sum( numpy.square(x) * invcov, axis=1 )
def g(x):
    return numpy.sum( x * invcov * mu, axis=1 )

b_dpt1 = []

for x in X:
    b_dpt1.append(f(x)-2.0*g(x))
    
### Pure python implementation ###
start = perf()
b_dpt2 = []
for cep in X:
    temp = []
    for row_invcov, row_mu in zip(invcov, mu):
        temp.append(0)
        for el_cep, el_invcov, el_mu in zip(cep, row_invcov, row_mu):
            temp[-1] += el_cep ** 2 * el_invcov - 2 * ( el_cep * el_invcov * el_mu )
    b_dpt2.append(temp)
end = perf()
naive_imple = end-start

### Numpy implementation ###
start = perf()
b_dpt = numpy.dot(numpy.square(X), invcov.T) - 2.0 * numpy.dot(X, numpy.transpose(mu * invcov))
end = perf()
numpy_imple = end-start

print(f"Naive implementation took {naive_imple/1000} microseconds\n"
      f"Numpy implementation took {numpy_imple/1000} microseconds\n"
      f"So, numpy here is {naive_imple/numpy_imple:.2f} times faster")

print(b_dpt - numpy.array(b_dpt2))

Naive implementation took 3670475.1 microseconds
Numpy implementation took 693.5 microseconds
So, numpy here is 5292.68 times faster
[[ 0.00000000e+00  7.10542736e-15 -2.13162821e-14 ... -7.10542736e-15
   0.00000000e+00  0.00000000e+00]
 [ 1.42108547e-14  7.10542736e-15  0.00000000e+00 ... -1.42108547e-14
   1.42108547e-14  1.42108547e-14]
 [ 0.00000000e+00 -1.42108547e-14  0.00000000e+00 ...  0.00000000e+00
  -7.10542736e-15  7.10542736e-15]
 ...
 [-1.42108547e-14 -7.10542736e-15  9.76996262e-15 ...  1.42108547e-14
   7.10542736e-15  0.00000000e+00]
 [ 0.00000000e+00  1.42108547e-14  1.42108547e-14 ...  0.00000000e+00
  -2.84217094e-14  2.84217094e-14]
 [ 1.77635684e-14 -2.13162821e-14  1.42108547e-14 ... -2.84217094e-14
   7.10542736e-15 -1.42108547e-14]]


## 1.2) Écrivez une fonction compute_lpp 

Cette méthode de la class **Mixture** va calculer les log posterior probabilités $$\log{p_i(x)}$$ d'un ensemble de vecteurs sur ce mélange de Gaussienne à matrices de covariances diagonales

Pour acumuler les statistiques, vous allez créer un objet mixture que vous appelerez **accum**.

* Cette méthode prend en paramètres une matrice de coefficients cepstraux de dimension N x F où N est le nombre de vecteurs (variable selon les fichiers) et F est la dimension des vecteurs (39 dans notre cas)
* En premier lieu, pensez à extraire des boucles tous les termes qui ne dépendent pas des données
* Cette méthode renvoie les $$n_i$$ (donc un vecteur)

Pour rappel: 
$$p_i(x) = \frac{1}{(2\pi)^{\frac{D}{2}}|\Sigma_i|^\frac{1}{2}} e^{-(\frac{1}{2}(x - \mu)^T\Sigma_i^{-1}(x-\mu))}$$
sachant:
$$ -(\frac{1}{2}(x - \mu_i)^T\Sigma_i^{-1}(x-\mu_i)) = -\frac{1}{2} ( x^T\Sigma_i^{-1}x + \mu_i^T\Sigma_i^{-1}\mu_i - x^T\Sigma_i^{-1}\mu_i - \mu_i^T\Sigma_i^{-1}x ) $$ 


In [56]:
# L'idée ici c'est de réutiliser les briques qu'on vient de construire ci-dessus pour calculer lpp

def compute_lpp_naive(X):
    # Compute the determinant
    det = []
    for line in invcov:
        det.append(1)
        for val in line:
            det[-1] *= val
        det[-1] = 1 / det[-1]
        
    # Compute the constant term
    cst = []
    for line in det:
        cst.append( 1.0 / ((2.0 * math.pi)**(D/2.0) * (math.sqrt(line))) )
    
    # Compute the independent term in the exponent:
    a_idpt = numpy.sum( numpy.square(mu) * invcov, axis=1 )
    
    # Calcul des parties dépendantes des données:
    b_dpt2 = []
    for cep in X:
        temp = []
        for row_invcov, row_mu in zip(invcov, mu):
            temp.append(0)
            for el_cep, el_invcov, el_mu in zip(cep, row_invcov, row_mu):
                temp[-1] += el_cep ** 2 * el_invcov - 2 * ( el_cep * el_invcov * el_mu )
        b_dpt2.append(temp)
    
    # Compute the log posterior probability
    lpp = []
    for row_b in b_dpt:
        temp = []
        for el_cst, el_a, el_b in zip(cst, a_idpt, row_b):
            temp.append( math.log(el_cst) - 0.5 * ( el_a + el_b ) )
        lpp.append(temp)
    
    return lpp


def compute_lpp_numpy(X):
    # Compute the determinant
    det = 1.0 / numpy.prod(invcov, axis=1)
        
    # Compute the constant term
    cst = 1.0 / ( numpy.power(2.0 * numpy.pi, D/2.0) * numpy.sqrt( det ))
    
    # Compute the independent term in the exponent:
    a_idpt = numpy.sum( numpy.square(mu) * invcov, axis=1 )
    
    # Calcul des parties dépendantes des données:
    b_dpt = numpy.dot(numpy.square(X), invcov.T) - 2.0 * numpy.dot(X, numpy.transpose(mu * invcov))
    
    # Compute the log posterior probability and return it
    
    return numpy.log(cst) - 0.5 * (a_idpt + b_dpt)

start = perf()
compute_lpp_naive(X)
end = perf()
naive_imple = end-start

start = perf()
compute_lpp_numpy(X)
end = perf()
numpy_imple = end-start

print(f"Naive implementation took {naive_imple/1000} microseconds\n"
      f"Numpy implementation took {numpy_imple/1000} microseconds\n"
      f"So, numpy here is {naive_imple/numpy_imple:.2f} times faster")

Naive implementation took 1741968.9 microseconds
Numpy implementation took 1097.5 microseconds
So, numpy here is 1587.22 times faster


## 1.3) Utilisez la fonction **sum_log_probabilities**
* Que fait cette fonction qui vous est donnée?
* Pourquoi l'utilise-t'on?

In [2]:
def sum_log_probabilities(lp):
    """Sum log probabilities in a secure manner to avoid extreme values

    :param lp: numpy array of log-probabilities to sum
    """
    pp_max = numpy.max(lp, axis=1)
    log_lk = pp_max + numpy.log(numpy.sum(numpy.exp((lp.transpose() - pp_max).T), axis=1))
    ind = ~numpy.isfinite(pp_max)
    if sum(ind) != 0:
        log_lk[ind] = pp_max[ind]
    pp = numpy.exp((lp.transpose() - log_lk).transpose())
    llk = log_lk.sum()
    return pp, llk

## 1.4) Bouclez sur les fichiers de paramètres pour accumuler les statistiques